## Fake News Classifier using Bidirectional LSTM RNN

#### Import Libraries

In [42]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense #Dense for creating neurons
from tensorflow.keras.layers import Bidirectional

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [26]:
tf.__version__

'2.10.0'

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Priya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Read the data

In [4]:
df = pd.read_csv("train.csv")
#df = pd.read_csv('train.csv',engine='python',error_bad_lines=False) #ParserError:Error tokenizing data. C error: EOF inside string starting at row 6675
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#### Summarize the data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [9]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
df = df.dropna()
df.shape

(18285, 5)

**Get the independent features**

In [13]:
X = df.drop('label',axis=1)

**Get the dependent features**

In [14]:
y = df['label']

**Check whether dataset is balanced or not**

In [15]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

#### Set the vocabulary size

In [21]:
voc_size = 5000

In [23]:
messages = X.copy()
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


**Dataset preprocessing**

In [31]:
messages.reset_index(inplace=True)

In [33]:
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [34]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

**One hot Encoding**

In [37]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[127, 1278, 1658, 188, 3225, 3462, 1408, 2364, 4167, 2149], [46, 4238, 829, 1927, 4233, 814, 2167], [1463, 536, 3977, 4694], [1859, 1273, 3839, 2069, 1746, 2687], [4059, 4233, 866, 2008, 2265, 834, 4233, 450, 632, 2947], [1352, 258, 2237, 4291, 3709, 3746, 4191, 4384, 3255, 3093, 4393, 2614, 1134, 2551, 2167], [538, 2199, 2770, 2718, 2956, 1860, 1243, 2897, 1247, 3905, 837], [87, 2582, 4315, 2716, 1231, 187, 3746, 1851, 1247, 3905, 837], [4755, 2296, 2537, 2750, 1832, 766, 517, 3524, 3746, 1365], [413, 2252, 4917, 1321, 2436, 3292, 4528, 4157], [2045, 1487, 2312, 3808, 1505, 4644, 4570, 3641, 4963, 3366, 3080], [2069, 4885, 3225, 766, 3746, 1231], [4726, 13, 193, 3420, 4072, 1851, 4588, 1837, 2657], [4275, 4759, 2615, 1312, 963, 4741, 2071, 1247, 3905, 837], [3792, 3972, 1866, 200, 1166, 1247, 3905, 837], [1073, 2733, 2522, 81, 3051, 2490, 4213, 1627, 4210, 2546], [3427, 2153, 4238], [2642, 3614, 4550, 2662, 3746, 1123, 4687, 2167], [618, 523, 829, 432, 2548, 852, 4202, 1347, 834], [2

**Embedding Representation : Converting words into vectors**

In [39]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding ='pre',maxlen = sent_length) 
embedded_docs

array([[   0,    0,    0, ..., 2364, 4167, 2149],
       [   0,    0,    0, ..., 4233,  814, 2167],
       [   0,    0,    0, ...,  536, 3977, 4694],
       ...,
       [   0,    0,    0, ..., 1247, 3905,  837],
       [   0,    0,    0, ..., 1780, 1997, 2460],
       [   0,    0,    0, ..., 2835, 4881, 4632]])

**Creating model**

In [41]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid')) #sigmoid because this is binary classification
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


**Convert Embedding_docs to array**

In [43]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

**Model training : Train test split**

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 27ms/step - loss: 0.3010 - accuracy: 0.8591 - val_loss: 0.1986 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1386 - accuracy: 0.9483 - val_loss: 0.2213 - val_accuracy: 0.9120
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0955 - accuracy: 0.9654 - val_loss: 0.2299 - val_accuracy: 0.9107
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0680 - accuracy: 0.9760 - val_loss: 0.2922 - val_accuracy: 0.9115
Epoch 5/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0503 - accuracy: 0.9833 - val_loss: 0.3218 - val_accuracy: 0.9137
Epoch 6/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0336 - accuracy: 0.9887 - val_loss: 0.3410 - val_accuracy: 0.9059
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0247 - accuracy: 0.9922 - val_loss: 0.4495 - val_accuracy: 0.904

**Performance metrics and accuracy**

In [45]:
y_pred = model.predict(X_test)

189/189 [==============================] - 3s 4ms/step


In [46]:
y_pred = np.where(y_pred > 0.5, 1, 0) #use AUC ROC Curve to get the threshold, here we considered it as 0.5

In [47]:
confusion_matrix(y_test,y_pred)

array([[3141,  278],
       [ 290, 2326]], dtype=int64)

In [48]:
accuracy_score(y_test,y_pred)

0.9058823529411765

In [49]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035

